<a href="https://colab.research.google.com/github/heesunpark1/Pozalabs_Assignment/blob/main/Pozalabs_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup Environment (필요 모듈 설치 및 라이브러리 불러오기)
import glob

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth2 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip3 install -q pyfluidsynth
!pip3 install -U magenta apache_beam


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 129504 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libinstpatch-1.0-2:amd64.
Preparing to unpack .../libinstpatch-1.0-2_1.1.2-2build1_amd64.deb ...
Unpacking libinstpatch-1.0-2:amd64 (1.1.2-2build1) ...
Selecting previously unselected package timgm6mb-soundfont.
Preparing to unpack .../timgm6mb-soundfont_1.3-3_all.deb ...
Unpacking timgm6mb-soundfont (1.3-3) ...
Selecting previously unselected package libfluidsynth2:amd64.
Preparing to unpack .../libfluidsynth2_2.1.1-2_amd64.deb ...
Unpacking libfluidsynth2:amd64 (2.1.1-2) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up timgm6mb-soundfont (1.3-3) ...
update-alternatives: using /usr/share/sounds/sf2/TimGM6mb.sf2 to provide /usr/share/sounds/sf2/default-GM.sf2 (default-GM.sf2)

Importing libraries and defining some helper functions...


/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term


Done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## [Preproc] Generate train tf data

In [ ]:
# 데이터 셋을 구글 드라이브에 저장 후 압축파일을 풀어줌

%cd /content/drive/MyDrive/DATA/pozlabs
! wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
! unzip -qq "/content/drive/MyDrive/DATA/pozlabs/groove-v1.0.0-midionly.zip"

/content/drive/MyDrive/DATA/pozlabs
--2023-01-24 03:56:19--  https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 142.250.145.128, 2a00:1450:4013:c14::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3260318 (3.1M) [application/zip]
Saving to: ‘groove-v1.0.0-midionly.zip.1’

groove-v1.0.0-midio 100%[===================>]   3.11M  --.-KB/s    in 0.08s   

2023-01-24 03:56:19 (37.7 MB/s) - ‘groove-v1.0.0-midionly.zip.1’ saved [3260318/3260318]

replace groove/drummer8/session2/25_latin_84_beat_4-4.mid? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
error:  cannot create groove/drummer8/session2/25_latin_84_beat_4-4.mid
        No such file or directory


In [ ]:
# 저장 경로 지정

INPUT_DIRECTORY="/content/drive/MyDrive/DATA/pozlabs/groove"
SEQUENCES_TFRECORD="/content/drive/MyDrive/DATA/pozlabs/notesequences.tfrecord"

In [ ]:
# TFrecord 형식으로 저장하는 전처리 진행 

%%bash -s "{INPUT_DIRECTORY}" "{SEQUENCES_TFRECORD}"

INPUT_DIRECTORY="${1}"
echo ${INPUT_DIRECTORY}

SEQUENCES_TFRECORD="${2}"
echo ${SEQUENCES_TFRECORD}

convert_dir_to_note_sequences \
  --input_dir=${INPUT_DIRECTORY} \
  --output_file=${SEQUENCES_TFRECORD} \
  --recursive

/content/drive/MyDrive/DATA/pozlabs/groove
/content/drive/MyDrive/DATA/pozlabs/notesequences.tfrecord


/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Converting files in '/content/drive/M

## [Train] Groovae_4bar MusicVAE Model

In [ ]:
# Music_VAE 모듈 불러오기 및 모델링을 위한 

import os

from magenta.models.music_vae import configs
from magenta.models.music_vae import data
import tensorflow.compat.v1 as tf
import tf_slim

tf.disable_v2_behavior()

flags = tf.app.flags

In [ ]:
flags.DEFINE_string(
    'master', '',
    'The TensorFlow master to use.')
flags.DEFINE_string(
    'examples_path', None,
    'Path to a TFRecord file of NoteSequence examples. Overrides the config.')
flags.DEFINE_string(
    'tfds_name', None,
    'TensorFlow Datasets dataset name to use. Overrides the config.')
flags.DEFINE_string(
    'run_dir', '/content/drive/MyDrive/DATA/pozlabs/',
    'Path where checkpoints and summary events will be located during '
    'training and evaluation. Separate subdirectories `train` and `eval` '
    'will be created within this directory.')
flags.DEFINE_integer(
    'num_steps', 50000,
    'Number of training steps or `None` for infinite.')
flags.DEFINE_integer(
    'eval_num_batches', None,
    'Number of batches to use during evaluation or `None` for all batches '
    'in the data source.')
flags.DEFINE_integer(
    'checkpoints_to_keep', 100,
    'Maximum number of checkpoints to keep in `train` mode or 0 for infinite.')
flags.DEFINE_integer(
    'keep_checkpoint_every_n_hours', 1,
    'In addition to checkpoints_to_keep, keep a checkpoint every N hours.')
flags.DEFINE_string(
    'mode', 'train',
    'Which mode to use (`train` or `eval`).')
flags.DEFINE_string(
    'config', '',
    'The name of the config to use.')
flags.DEFINE_string(
    'hparams', '',
    'A comma-separated list of `name=value` hyperparameter values to merge '
    'with those in the config.')
flags.DEFINE_bool(
    'cache_dataset', True,
    'Whether to cache the dataset in memory for improved training speed. May '
    'cause memory errors for very large datasets.')
flags.DEFINE_integer(
    'task', 0,
    'The task number for this worker.')
flags.DEFINE_integer(
    'num_ps_tasks', 0,
    'The number of parameter server tasks.')
flags.DEFINE_integer(
    'num_sync_workers', 0,
    'The number of synchronized workers.')
flags.DEFINE_string(
    'eval_dir_suffix', '',
    'Suffix to add to eval output directory.')
flags.DEFINE_string(
    'log', 'DEBUG',
    'The threshold for what messages will be logged: '
    'DEBUG, INFO, WARN, ERROR, or FATAL.')
flags.DEFINE_string("f", "", "kernel")

In [ ]:

flags.FLAGS.run_dir = '/content/drive/MyDrive/DATA/pozlabs/'
flags.FLAGS.config = 'groovae_4bar'
flags.FLAGS.examples_path = '/content/drive/MyDrive/DATA/pozlabs/notesequences.tfrecord'
# flags.FLAGS.tfds_name = 'groove/full-midionly'
flags.FLAGS.log = "INFO"
# flags.FLAGS.hparams = 

In [ ]:
FLAGS = flags.FLAGS

tf.logging.set_verbosity(FLAGS.log)

print(flags.FLAGS.config)

groovae_4bar


In [ ]:
# 써머리 텐서보드 함수
def _trial_summary(hparams, examples_path, output_dir):
    """Writes a tensorboard text summary of the trial."""

    examples_path_summary = tf.summary.text(
        'examples_path', tf.constant(examples_path, name='examples_path'),
        collections=[])

    hparams_dict = hparams.values()

    # 하이퍼 파라미타
    # Create a markdown table from hparams.
    header = '| Key | Value |\n| :--- | :--- |\n'
    keys = sorted(hparams_dict.keys())
    lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
    hparams_table = header + '\n'.join(lines) + '\n'

    hparam_summary = tf.summary.text(
        'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
        writer.add_summary(examples_path_summary.eval())
        writer.add_summary(hparam_summary.eval())
        writer.close()

# 텐서를 입력값으로 넣는 함수
def _get_input_tensors(dataset, config):
    """Get input tensors from dataset."""
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence,
    sequence_length) = iterator.get_next()

    input_sequence.set_shape(
        [batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape(
        [batch_size, None, config.data_converter.output_depth])
    
    if not config.data_converter.control_depth:
        control_sequence = None
    else:
        control_sequence.set_shape(
        [batch_size, None, config.data_converter.control_depth])

    sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

#훈련 함수 
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
    """Train loop."""
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    if is_chief:
        _trial_summary(
            config.hparams, config.train_examples_path or config.tfds_name, train_dir)
        
    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
            num_ps_tasks, merge_devices=True)):

            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)

            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,
                    num_sync_workers)
                
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)

            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))

            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step')

            logging_dict = {'global_step': model.global_step,
                      'loss': model.loss}

            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))

            scaffold = tf.train.Scaffold(
                    saver=tf.train.Saver(
                        max_to_keep=checkpoints_to_keep,
                        keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours
                    ))
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)
# 검증 함수
def evaluate(train_dir,
             eval_dir,
             config,
             dataset_fn,
             num_batches,
             master=''):
    """Evaluate the model repeatedly."""
    tf.gfile.MakeDirs(eval_dir)

    _trial_summary(
        config.hparams, config.eval_examples_path or config.tfds_name, eval_dir)
    
    with tf.Graph().as_default():
        model = config.model
        model.build(config.hparams,
                    config.data_converter.output_depth,
                    is_training=False)

        eval_op = model.eval(
            **_get_input_tensors(dataset_fn().take(num_batches), config))

        hooks = [
            tf_slim.evaluation.StopAfterNEvalsHook(num_batches),
            tf_slim.evaluation.SummaryAtEndHook(eval_dir)
        ]

        tf_slim.evaluation.evaluate_repeatedly(
            train_dir,
            eval_ops=eval_op,
            hooks=hooks,
            eval_interval_secs=60,
            master=master)

In [ ]:
if not FLAGS.run_dir:
    raise ValueError('Invalid run directory: %s' % FLAGS.run_dir)
run_dir = os.path.expanduser(FLAGS.run_dir)
train_dir = os.path.join(run_dir, 'train')

if FLAGS.mode not in ['train', 'eval']:
    raise ValueError('Invalid mode: %s' % FLAGS.mode)

if FLAGS.config not in configs.CONFIG_MAP:
    raise ValueError('Invalid config: %s' % FLAGS.config)

config = configs.CONFIG_MAP[FLAGS.config]

if FLAGS.hparams:
    config.hparams.parse(FLAGS.hparams)
config_update_map = {}

if FLAGS.examples_path:
    config_update_map['%s_examples_path' % FLAGS.mode] = os.path.expanduser(FLAGS.examples_path)

if FLAGS.tfds_name:
    if FLAGS.examples_path:
        raise ValueError('At most one of --examples_path and --tfds_name can be set.')

    config_update_map['tfds_name'] = FLAGS.tfds_name
    config_update_map['eval_examples_path'] = None
    config_update_map['train_examples_path'] = None

config = configs.update_config(config, config_update_map)

if FLAGS.num_sync_workers:
    config.hparams.batch_size //= FLAGS.num_sync_workers

def dataset_train_fn():
    return data.get_dataset(
        config,
        tf_file_reader=tf.data.TFRecordDataset,
        is_training=True,
        cache_dataset=FLAGS.cache_dataset)

def dataset_eval_fn():
    return data.get_dataset(
        config,
        tf_file_reader=tf.data.TFRecordDataset,
        is_training=False,
        cache_dataset=FLAGS.cache_dataset)

In [ ]:
#훈련(학습) 진행

train(
    train_dir,
    config=config,
    dataset_fn=dataset_train_fn,
    checkpoints_to_keep=FLAGS.checkpoints_to_keep,
    keep_checkpoint_every_n_hours=FLAGS.keep_checkpoint_every_n_hours,
    num_steps=FLAGS.num_steps,
    master=FLAGS.master,
    num_sync_workers=FLAGS.num_sync_workers,
    num_ps_tasks=FLAGS.num_ps_tasks,
    task=FLAGS.task
)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 512, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

INFO:tensorflow:Reading examples from file: /content/drive/MyDrive/DATA/pozlabs/notesequences.tfrecord
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is

In [ ]:
num_batches = FLAGS.eval_num_batches or data.count_examples(
    config.eval_examples_path,
    config.tfds_name,
    config.data_converter,
    file_reader) // config.hparams.batch_size

eval_dir = os.path.join(run_dir, 'eval' + FLAGS.eval_dir_suffix)

evaluate(
    train_dir,
    eval_dir,
    config=config,
    dataset_fn=dataset_eval_fn,
    num_batches=num_batches,
    master=FLAGS.master)

NameError: ignored

In [ ]:
# 모델링

from magenta.models.music_vae.trained_model import TrainedModel

model = TrainedModel(
    configs.CONFIG_MAP['groovae_4bar'],  # 4마디 샘플 지정
    batch_size=1, 
    checkpoint_dir_or_path='/content/drive/MyDrive/DATA/pozlabs/train' #학습시킨 체크포인트 경로
)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

/usr/local/lib/python3.8/dist-packages/magenta/models/music_vae/lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
/usr/local/lib/python3.8

In [ ]:
# 샘플 생성
import note_seq

generated_sequence = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(generated_sequence[0], '/content/drive/MyDrive/DATA/pozlabs/generated_drum_4bar_2.mid')